In [1]:
import os
# loading from a .env file
# load_dotenv(dotenv_path="/full/path/to/your/.env")

# or 
# if you're on google colab just uncomment below and replace with your openai api key
# os.environ["OPENAI_API_KEY"] = "<your-openai-api-key>"

In [2]:
MODEL = 'gpt-4o-mini'

In [3]:
from openai import OpenAI
client = OpenAI()

def recommender_chat_bot(user_query):
    response = client.chat.completions.create(
        model=MODEL,
        messages=[{"role": "system", "content": "You are a chatbot system \
            and recommender for products about tech to a diverse group of users."},
                  {"role": "user", "content": user_query}]
    )
    
    return response.choices[0].message.content

In [4]:
def generate_data_with_chatgpt(data_example):
    response = client.chat.completions.create(
        model=MODEL,
        messages=[{"role": "system", "content": "You generate examples of tech products."},
                  {"role": "user", "content": data_example}]
    )
    
    return response.choices[0].message.content


tech_products_prompt = "Create 20 examples of tech products\
    with their descriptions as a list of bullet points"

data_tech = generate_data_with_chatgpt(tech_products_prompt)
data_tech

'Sure! Here are 20 examples of tech products along with their descriptions:\n\n1. **Smartphone with AI Assistant**  \n   - A high-performance smartphone equipped with an integrated AI assistant that learns user habits, manages schedules, and offers personalized recommendations.\n\n2. **Wireless Noise-Canceling Headphones**  \n   - Over-ear headphones that utilize active noise cancellation technology to block out external sounds, providing an immersive listening experience.\n\n3. **Smart Home Security Camera**  \n   - A Wi-Fi-enabled camera that allows users to monitor their homes in real-time via a smartphone app, featuring motion detection and night vision capabilities.\n\n4. **Portable Solar Charger**  \n   - A compact solar panel that can charge multiple devices simultaneously, perfect for outdoor activities and travel, equipped with multiple USB ports.\n\n5. **Fitness Tracking Smartwatch**  \n   - A smartwatch designed to monitor health metrics like heart rate, step count, and calo

In [5]:
from IPython.display import Markdown
Markdown(data_tech)

Sure! Here are 20 examples of tech products along with their descriptions:

1. **Smartphone with AI Assistant**  
   - A high-performance smartphone equipped with an integrated AI assistant that learns user habits, manages schedules, and offers personalized recommendations.

2. **Wireless Noise-Canceling Headphones**  
   - Over-ear headphones that utilize active noise cancellation technology to block out external sounds, providing an immersive listening experience.

3. **Smart Home Security Camera**  
   - A Wi-Fi-enabled camera that allows users to monitor their homes in real-time via a smartphone app, featuring motion detection and night vision capabilities.

4. **Portable Solar Charger**  
   - A compact solar panel that can charge multiple devices simultaneously, perfect for outdoor activities and travel, equipped with multiple USB ports.

5. **Fitness Tracking Smartwatch**  
   - A smartwatch designed to monitor health metrics like heart rate, step count, and calories burned, paired with a companion app for detailed analysis.

6. **VR Headset for Gaming**  
   - A virtual reality headset that offers a fully immersive gaming experience with high-resolution displays and motion tracking for realistic gameplay.

7. **Wireless Charging Pad**  
   - A sleek, minimal design charging pad that supports Qi-enabled devices, providing fast charging without the need for cables or plugs.

8. **Bluetooth Smart Lock**  
   - A keyless entry lock that can be controlled remotely via a smartphone app, allowing users to lock/unlock their doors from anywhere and grant guest access.

9. **Smart Refrigerator with Touchscreen**  
   - An innovative refrigerator that features a touchscreen interface, allowing for recipe suggestions, grocery list management, and entertainment options.

10. **Drones with 4K Camera**  
    - A high-flying drone equipped with a 4K camera for stunning aerial photography and video recording, featuring GPS and smart flight modes.

11. **Home Automation Hub**  
    - A central device that connects and controls various smart home products, allowing users to create automation routines and manage devices through a single app.

12. **E-Reader with Adjustable Backlight**  
    - A lightweight electronic book reader featuring an adjustable warm and cool backlight, perfect for reading day or night without straining the eyes.

13. **Streaming Media Player**  
    - A compact device that plugs into your TV, enabling access to popular streaming services with 4K resolution and voice search functionality.

14. **Wireless Earbuds with Touch Control**  
    - True wireless earbuds that offer touch-sensitive controls for music playback and calls, featuring sweat resistance for an active lifestyle.

15. **Home 3D Printer**  
    - A desktop 3D printer designed for hobbyists and makers, capable of printing various materials to create custom objects, prototypes, and designs.

16. **Digital Photo Frame with Wi-Fi**  
    - A digital frame that displays a slideshow of photos uploaded via Wi-Fi, allowing family and friends to share images remotely.

17. **Smart Thermostat**  
    - An energy-efficient thermostat that learns user preferences over time, optimizing heating and cooling schedules to save on energy costs.

18. **Augmented Reality Glasses**  
    - Lightweight glasses that overlay digital information on the real world, enriching experiences in navigation, gaming, and education.

19. **AI-Powered Language Translation Device**  
    - A portable device that translates languages in real-time, ideal for travelers and business professionals, equipped with voice recognition technology.

20. **Robot Vacuum Cleaner**  
    - An autonomous vacuum cleaner that uses smart sensors to navigate and clean floors efficiently, programmable via a smartphone app for scheduled cleaning.

Feel free to ask if you need more information or additional examples!

In [6]:
def create_costumer_profile(user_query):
    response = client.chat.completions.create(
        model=MODEL,
        messages=[{"role": "system", "content": "You create a list of customer \
            profiles based on input."},
                  {"role": "user", "content": user_query}]
    )
    
    return response.choices[0].message.content


create_costumer_profile = create_costumer_profile("Create 10 customer profiles \
    for users with different backgrounds and preferences.")

Markdown(create_costumer_profile)

Sure! Here are 10 diverse customer profiles based on varying backgrounds and preferences:

### Customer Profile 1: Sarah Thompson
- **Age**: 28
- **Occupation**: Graphic Designer
- **Location**: Austin, TX
- **Interests**: Art, DIY projects, sustainability
- **Shopping Preferences**: Prefers eco-friendly products and unique handmade items. Active on social media, often posts about her projects.
- **Spending Behavior**: Moderate spender, willing to pay more for sustainability and aesthetics.

### Customer Profile 2: John Patel
- **Age**: 45
- **Occupation**: IT Manager
- **Location**: San Francisco, CA
- **Interests**: Technology, gaming, outdoor activities
- **Shopping Preferences**: Values performance and quality in tech products. Often looks for the latest gadgets and gaming equipment.
- **Spending Behavior**: High spender, especially on tech-related items, loyal to brands he trusts.

### Customer Profile 3: Maria Gonzalez
- **Age**: 34
- **Occupation**: School Teacher
- **Location**: Miami, FL
- **Interests**: Education, reading, parenting
- **Shopping Preferences**: Seeks educational tools and books for her classroom and family. Enjoys discounts and bulk purchases.
- **Spending Behavior**: Budget-conscious but willing to invest in quality educational materials.

### Customer Profile 4: David Chen
- **Age**: 22
- **Occupation**: College Student
- **Location**: New York City, NY
- **Interests**: Fashion, social media, music
- **Shopping Preferences**: Follows fashion trends, shop mainly at online stores, enjoys thrifting. Looks for the latest styles at affordable prices.
- **Spending Behavior**: Low to moderate spender, often waits for sales or discounts.

### Customer Profile 5: Emily Carter
- **Age**: 50
- **Occupation**: Nurse
- **Location**: Seattle, WA
- **Interests**: Health, cooking, gardening
- **Shopping Preferences**: Prefers organic food and health-related products. Often shops at local markets and values nutritional information.
- **Spending Behavior**: Moderate spender, focuses on health and quality rather than price alone.

### Customer Profile 6: Tom Robinson
- **Age**: 40
- **Occupation**: Construction Worker
- **Location**: Denver, CO
- **Interests**: Sports, home improvement, BBQ
- **Shopping Preferences**: Looks for durable workwear and tools. Enjoys deals at home improvement stores and warehouse clubs.
- **Spending Behavior**: Moderate to high spender, prioritizes practical and quality products.

### Customer Profile 7: Anjali Sharma
- **Age**: 30
- **Occupation**: Marketing Consultant
- **Location**: Chicago, IL
- **Interests**: Travel, wellness, technology
- **Shopping Preferences**: Invests in travel gear and wellness products. Enjoys premium brands and experiences.
- **Spending Behavior**: High spender, especially on travel and luxury items.

### Customer Profile 8: Richard Johnson
- **Age**: 55
- **Occupation**: Retired
- **Location**: Orlando, FL
- **Interests**: Golf, traveling, volunteering
- **Shopping Preferences**: Values comfort and ease in products. Enjoys shopping for hobbies and leisure activities.
- **Spending Behavior**: Moderate spender, often looking for value and quality in leisure-related purchases.

### Customer Profile 9: Lily Nguyen
- **Age**: 26
- **Occupation**: Software Engineer
- **Location**: Toronto, Canada
- **Interests**: Coding, gaming, movies
- **Shopping Preferences**: Loves tech and gaming products, often participates in online gaming communities. Enjoys sci-fi and fantasy merchandise.
- **Spending Behavior**: Moderate to high spender on gaming equipment and tech upgrades.

### Customer Profile 10: Carlos Ramirez
- **Age**: 38
- **Occupation**: Restaurant Owner
- **Location**: San Diego, CA
- **Interests**: Culinary arts, wine tasting, fitness
- **Shopping Preferences**: Seeks gourmet cooking supplies and health-conscious food options. Enjoys premium quality products.
- **Spending Behavior**: High spender, especially in gourmet and culinary-related categories.

These profiles illustrate the complexity of customer preferences, behaviors, and demographics.

In [7]:
profile_10 = """
Customer Profile 10: Carlos Ramirez
Age: 38
Occupation: Restaurant Owner
Location: San Diego, CA
Interests: Culinary arts, wine tasting, fitness
Shopping Preferences: Seeks gourmet cooking supplies and health-conscious food options. Enjoys premium quality products.
Spending Behavior: High spender, especially in gourmet and culinary-related categories.
These profiles illustrate the complexity of customer preferences, behaviors, and demographics.
"""


def recommend_product(profile):
    prompt_to_recommender = f"Recommend a tech product from this list ONLY:\n\n{data_tech} \
        for this customer profile: {profile}.\
        Output ONLY A SIMPLE MESSAGE REFERENCING THE CUSTOMER AND THE recommendations."
    
    recommendation = recommender_chat_bot(prompt_to_recommender)
    
    return recommendation


product_recommendation = recommend_product(profile_10)

Markdown(product_recommendation)

Hi Carlos! Based on your interests and preferences, I recommend the **Smart Refrigerator with Touchscreen**. It can help you manage your ingredients, suggest recipes, and keep your culinary essentials organized, enhancing your gourmet cooking experience. Enjoy!

In [9]:
data_tech

'Sure! Here’s a list of 20 tech products along with their descriptions:\n\n1. **Smartphone 12X Pro**  \n   A flagship smartphone featuring a 6.9-inch AMOLED display, a quad-camera system with 108 MP main sensor, and AI enhancements for photography.\n\n2. **EcoSmart Home Hub**  \n   A central device for controlling all smart home gadgets, compatible with Alexa, Google Home, and Apple HomeKit, enabling seamless automation and energy monitoring.\n\n3. **Wireless Noise-Canceling Headphones**  \n   Over-ear headphones with adaptive noise cancellation, 30 hours of playback time, and touch controls for music and calls.\n\n4. **4K Ultra HD Streaming Device**  \n   A compact streaming device that supports all major services like Netflix and Hulu, including voice search and a customizable interface.\n\n5. **Fitness Tracker 2.0**  \n   A sleek wristband that monitors heart rate, sleep patterns, and calories burned, featuring GPS for outdoor activities and waterproof design.\n\n6. **Portable Solar

In [18]:
from pydantic import BaseModel, Field
from typing import List


class TechProduct(BaseModel):
    names: List[str] = Field(..., title="All the product names", description="The titles of the products in question")
    descriptions: List[str] = Field(..., title="All the product descriptions", description="A complete list of descriptions of the products extracted from the source.")
    
def extract_tech_product_data(raw_tech_data: str):
    response = client.beta.chat.completions.parse(
        model="gpt-4o",
        messages=[
            {'role': 'system', 'content': 'You are an extraction engine for tech products, \
                you extract the following fields:\
                1. name - list of all names of products in the input\
                2. description - list of all descritionps in the products\
                    Make sure to extract all the names and descriptions from the input.'},
        ],
        response_format=TechProduct,
        temperature=0.0
    )
    
    return response.choices[0].message

tech_data_structured = extract_tech_product_data(f'Extract all the names and descriptions from this raw data: \n\n {data_tech}')
tech_data_structured

ParsedChatCompletionMessage[TechProduct](content='{"names":["iPhone 15","Samsung Galaxy S23","MacBook Pro 2023","Sony WH-1000XM5","Google Pixel Watch"],"descriptions":["The iPhone 15 features a new design, improved camera system, and the latest A17 Bionic chip for enhanced performance.","Samsung Galaxy S23 offers a stunning display, powerful processor, and long-lasting battery life, making it a top choice for Android users.","MacBook Pro 2023 comes with the latest M3 chip, offering unprecedented speed and efficiency for professionals and creatives alike.","Sony WH-1000XM5 headphones provide industry-leading noise cancellation, superior sound quality, and all-day comfort.","Google Pixel Watch integrates seamlessly with Android devices, offering health tracking, notifications, and a sleek design."]}', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=[], parsed=TechProduct(names=['iPhone 15', 'Samsung Galaxy S23', 'MacBook Pro 2023', 'Sony WH-1000XM5', 'Google Pi

In [20]:
tech_data_structured.parsed.names

['iPhone 15',
 'Samsung Galaxy S23',
 'MacBook Pro 2023',
 'Sony WH-1000XM5',
 'Google Pixel Watch']

In [22]:
tech_data_structured.parsed.descriptions

['The iPhone 15 features a new design, improved camera system, and the latest A17 Bionic chip for enhanced performance.',
 'Samsung Galaxy S23 offers a stunning display, powerful processor, and long-lasting battery life, making it a top choice for Android users.',
 'MacBook Pro 2023 comes with the latest M3 chip, offering unprecedented speed and efficiency for professionals and creatives alike.',
 'Sony WH-1000XM5 headphones provide industry-leading noise cancellation, superior sound quality, and all-day comfort.',
 'Google Pixel Watch integrates seamlessly with Android devices, offering health tracking, notifications, and a sleek design.']

In [23]:
def create_diagrams_with_AI(user_query):
    response = client.chat.completions.create(
        model=MODEL,
        messages=[{"role": "system", "content": "You create mermaid diagrams with AI."},
                  {"role": "user", "content": user_query}]
    )
    
    return response.choices[0].message.content

output_diagram_raw = create_diagrams_with_AI("Create a diagram of a flowchart that shows the process of a customer \
    buying a product from an e-commerce website.")

Markdown(output_diagram_raw)

Here's a mermaid diagram representing a flowchart for the process of a customer buying a product from an e-commerce website:

```mermaid
flowchart TD
    A[Start] --> B[Visit E-commerce Website]
    B --> C[Browse Products]
    C --> D{Select Product?}
    D -->|Yes| E[Add Product to Cart]
    D -->|No| C
    E --> F[View Cart]
    F --> G{Proceed to Checkout?}
    G -->|Yes| H[Enter Shipping Information]
    G -->|No| C
    H --> I[Choose Payment Method]
    I --> J[Enter Payment Information]
    J --> K[Review Order]
    K --> L{Confirm Order?}
    L -->|Yes| M[Place Order]
    L -->|No| F
    M --> N[Receive Order Confirmation]
    N --> O[End]
```

This diagram outlines the steps a customer takes from visiting an e-commerce website to receiving an order confirmation after purchasing a product.

In [29]:
class MermaidDiagram(BaseModel):
    diagram: str = Field(..., title="Mermaid Diagram", description="The mermaid diagram as a properly formatted markdown ready to visualize.")

def create_diagrams_with_AI(user_query):
    response = client.beta.chat.completions.parse(
        model=MODEL,
        messages=[{"role": "system", "content": "You create mermaid diagrams with AI ready to be visualized in markdown."},
                  {"role": "user", "content": user_query}],
        response_format=MermaidDiagram,
        temperature=0.0
    )
    
    return response.choices[0].message.parsed.diagram

output_diagram_raw = create_diagrams_with_AI("Create a diagram of a flowchart that shows the process of a customer \
    buying a product from an e-commerce website.")

Markdown(output_diagram_raw)

```mermaid
flowchart TD
    A[Start] --> B[Visit E-commerce Website]
    B --> C[Browse Products]
    C --> D[Select Product]
    D --> E[Add to Cart]
    E --> F[View Cart]
    F --> G[Proceed to Checkout]
    G --> H[Enter Shipping Information]
    H --> I[Enter Payment Information]
    I --> J[Review Order]
    J --> K[Place Order]
    K --> L[Receive Confirmation]
    L --> M[End]
```